In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os

In [3]:
import bundesrat

In [4]:
USE_CACHE = True

In [5]:
if USE_CACHE:
    os.makedirs('./_cache', exist_ok=True)

In [6]:
sessions = []
if os.path.exists('sessions.json'):
    with open('sessions.json') as f:
        sessions = json.load(f)

In [7]:
if not sessions:
    sessions = (
        #list(bundesrat.get_sessions_this_year(cache=USE_CACHE)) + #By now, archive also includes the sessions of the current year
        list(bundesrat.get_sessions_archive(cache=USE_CACHE))
    )
    with open('sessions.json', 'w') as f:
        json.dump(sessions, f)

In [8]:
for session in sessions:
    if 'tops' in session:
        continue
    print('\nLoading tops of: %s' % session['number'])

    session['tops'] = list(bundesrat.get_session_tops(session['number']))
    with open('sessions.json', 'w') as f:
        json.dump(sessions, f)


Loading tops of: 992
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23a 23b 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40a 40b 40c 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69a 69b 69c 69d 70a 70b 71 72 73 74 75a 75b 76 77 78 79 80 81 82 83 84 85 86 87a 87b 88 999a 999b 999c 999d 999e 
Loading tops of: 991
1a 1b 1c 
Loading tops of: 990
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32a 32b 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 
Loading tops of: 989
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35a 35b 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50a 50b 50c 50d 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77a 77b 
Loading tops of: 988
1a 1b 1c 1d 1e 1f 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24a 24b 24c 24d 25 26 999a 
Loading tops of: 987
1 
Loading tops of: 986
1 2 3 4 5 6 7 8 9 10 11 12 1

KeyboardInterrupt: 